In [68]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import matplotlib

In [69]:

def svd_decomposition(feature_space, feature):
   a = []
   for element in feature_space:
      a.append(element[feature])

   a_array = np.array(a)

   U, S, Vh = np.linalg.svd(a_array, full_matrices=False, compute_uv=True, hermitian=False)

   return U, S, Vh


def get_top_latent_features(U, S, threshold):
   top_k_feature = np.array([elem for elem in S if elem > threshold])

   new_k = len(top_k_feature)

   return U[:, :new_k]

   

def plot_latent_features(S):
   S_list = S.tolist()

   top_k_list = [elem for elem in S_list if elem > 100000]

   plt.plot(S_list, marker='o')  # 'marker' aggiunge i punti ai valori
   plt.title('Andamento continuo dei dati')
   plt.xlabel('Indice')
   plt.ylabel('Valore')
   plt.grid(True)
   plt.show()


   print(top_k_list)
   print(len(top_k_list))



In [70]:
def serialize_latent_space(feature_space, U, path="/workspaces/dbm25/data/svd_latent_features.pt"):
   latent_feature_list = U.tolist()

   old_feature_space = pd.DataFrame(feature_space)
   old_feature_space = old_feature_space[["file_path", "class"]]
   old_feature_space["svd_latent"] = latent_feature_list

   new_feature_space = []

   for index, elem in old_feature_space.iterrows():
      new_feature_space.append(elem.to_dict())

   torch.save(new_feature_space, f=path)

In [71]:
#feature_space = torch.load("/workspaces/dbm25/data/extracted_features.pt")

feature = "hog"
threshold = 100000



# U, S, Vh = svd_decomposition(feature_space, feature)
# print(f"U.shape: {U.shape}, S.shape: {S.shape}, Vh.shape: {Vh.shape}")

#new_U = get_top_latent_features(U, S, threshold)

top_k_latent_feature_space = torch.load("/workspaces/dbm25/data/svd_top_k_latent_features.pt")
latent_feature_space = torch.load("/workspaces/dbm25/data/svd_latent_features.pt")

print(top_k_latent_feature_space[0]["svd_latent"])
print(latent_feature_space[0]["svd_latent"])

weight_list = []
"""
print(latent_feature_space[0])
for element in latent_feature_space:
   index_weight = [(i, weight) for i, weight in enumerate(element["svd_latent"])]
   #sorted_weights = np.sort(element["svd_latent"])[::-1]
   index_weight = sorted(index_weight, key=lambda x: x[1], reverse=True)
   image_id = element["file_path"].split("/").pop()
   new_pair = (image_id, index_weight)
   print(new_pair)
   break
"""

 



   

#plot_latent_features(S)


# test
#print(new_U[0] == U[0][:5])



[-0.016673724597784628, -0.010154802233786868, -0.009678301061193564, 0.0045840243409848895, -0.007789152347961918]
[-0.016673724597784628, -0.010154802233786868, -0.009678301061193564, 0.0045840243409848895, -0.007789152347961918, -0.009789932774281305, -0.015898522651757837, 0.007813018349134778, 0.012947843189133564, -0.008535907198625098, 0.022267003272862993, 0.0050845687437505145, -0.0009617652587552818, 0.0017734909009584432, -0.01927064714730169, 0.018750588877928066, 0.010195627293968907, -0.014811497134564405, -0.017313145173645288, -0.0024528830763628226, 0.006428312048048199, -0.010261608954019256, 0.00938306858419813, -0.0008848165295434401, -0.00012317918662769047, -0.0017638030022707665, 0.0004552643854857938, 0.005143947051331034, 0.005222683414431162, -0.016350457845825145, -0.005762882485700027, 0.02106213429556215, 0.0009282861818511777, -0.011610604629621449, -0.011538997869968058, -0.00924258747070654, 0.00223472937582133, -0.0019251186276628414, 0.0104630166880956

'\nprint(latent_feature_space[0])\nfor element in latent_feature_space:\n   index_weight = [(i, weight) for i, weight in enumerate(element["svd_latent"])]\n   #sorted_weights = np.sort(element["svd_latent"])[::-1]\n   index_weight = sorted(index_weight, key=lambda x: x[1], reverse=True)\n   image_id = element["file_path"].split("/").pop()\n   new_pair = (image_id, index_weight)\n   print(new_pair)\n   break\n'